In [ ]:
import numpy as np

In [ ]:
messages_texts = np.load('./extracted_messages.npy')

# Pre-processing

## Language Detection

In [3]:
import spacy
from spacy_langdetect import LanguageDetector
nlp = spacy.load('en')
nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)

OSError: [E941] Can't find model 'en'. It looks like you're trying to load a model from a shortcut, which is obsolete as of spaCy v3.0. To load the model, use its full name instead:

nlp = spacy.load("en_core_web_sm")

For more details on the available models, see the models directory: https://spacy.io/models. If you want to create a blank model, use spacy.blank: nlp = spacy.blank("en")

In [24]:
doc = [nlp(str(messages_text))._.language for messages_text in messages_texts]

TypeError: ignored